In [1]:
!pip install ttkthemes



In [2]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class BMICalculator:
    def __init__(self, root):
        self.root = root
        self.root.title("BMI Calculator")

        # Set dark theme appearance
        root.configure(bg="#2E2E2E")  # Dark background color
        label_color = "white"  # Text color

        # Center-align the frame
        self.root.geometry("300x300")
        self.root.update_idletasks()
        width = self.root.winfo_width()
        height = self.root.winfo_height()
        x_offset = (self.root.winfo_screenwidth() - width) // 2
        y_offset = (self.root.winfo_screenheight() - height) // 2
        self.root.geometry(f"+{x_offset}+{y_offset}")

        # GUI components
        self.weight_label = tk.Label(root, text="Weight (kg):", bg="#2E2E2E", fg=label_color)
        self.weight_entry = tk.Entry(root, bg="#2E2E2E", fg=label_color)
        self.weight_label.pack(pady=10)
        self.weight_entry.pack(pady=10)

        self.height_label = tk.Label(root, text="Height (m):", bg="#2E2E2E", fg=label_color)
        self.height_entry = tk.Entry(root, bg="#2E2E2E", fg=label_color)
        self.height_label.pack(pady=10)
        self.height_entry.pack(pady=10)

        self.calculate_button = tk.Button(root, text="Calculate BMI", command=self.calculate_bmi, bg="#4682B4", fg="white")
        self.calculate_button.pack(pady=20)

        self.view_history_button = tk.Button(root, text="View History", command=self.view_history, bg="#4682B4", fg="white")
        self.view_history_button.pack(pady=10)

        self.plot_button = tk.Button(root, text="Plot BMI Trend", command=self.plot_bmi_trend, bg="#4682B4", fg="white")
        self.plot_button.pack(pady=10)

        # Database setup
        self.conn = sqlite3.connect('bmi_data.db')
        self.cursor = self.conn.cursor()
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS bmi_records (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                weight REAL,
                height REAL,
                bmi REAL
            )
        ''')
        self.conn.commit()

    def calculate_bmi(self):
        try:
            # Validate user inputs
            weight = float(self.weight_entry.get())
            height = float(self.height_entry.get())

            if not (0 < weight < 500) or not (0.5 < height < 3):
                raise ValueError("Invalid input values. Please enter reasonable values for weight and height.")

            bmi = weight / (height ** 2)

            # Save data to the database
            self.cursor.execute("INSERT INTO bmi_records (weight, height, bmi) VALUES (?, ?, ?)", (weight, height, bmi))
            self.conn.commit()

            # Categorize BMI
            category = self.categorize_bmi(bmi)

            messagebox.showinfo("BMI Result", f"Your BMI is: {bmi:.2f}\nCategory: {category}")

        except ValueError as e:
            messagebox.showerror("Error", str(e))

    def categorize_bmi(self, bmi):
        if bmi < 18.5:
            return "Underweight"
        elif 18.5 <= bmi < 25:
            return "Normal weight"
        elif 25 <= bmi < 30:
            return "Overweight"
        else:
            return "Obese"

    def view_history(self):
        history_window = tk.Toplevel(self.root)
        history_window.title("BMI History")

        self.cursor.execute("SELECT * FROM bmi_records")
        records = self.cursor.fetchall()

        for i, record in enumerate(records):
            tk.Label(history_window, text=f"Entry {i + 1}: Weight: {record[1]}, Height: {record[2]}, BMI: {record[3]:.2f}", bg="#2E2E2E", fg="white").pack()

    def plot_bmi_trend(self):
        self.cursor.execute("SELECT * FROM bmi_records")
        records = self.cursor.fetchall()

        if not records:
            messagebox.showinfo("Info", "No data available for plotting.")
            return

        weights = [record[1] for record in records]
        heights = [record[2] for record in records]
        bmis = [record[3] for record in records]

        fig, ax = plt.subplots()
        ax.plot(bmis, label='BMI Trend')
        ax.set_xlabel('Entries')
        ax.set_ylabel('BMI')
        ax.legend()

        # Create a Tkinter canvas containing the Matplotlib figure
        canvas = FigureCanvasTkAgg(fig, master=self.root)
        canvas_widget = canvas.get_tk_widget()

        # Pack the canvas into the Tkinter window
        canvas_widget.pack()

if __name__ == "__main__":
    root = tk.Tk()
    app = BMICalculator(root)
    root.mainloop()
